In [22]:
from pymatgen.io.vasp import Poscar
from pymatgen.io.lammps.data import LammpsData


def remove_masses_section_from_text(data_text):
    """
    根据原子类型数删除 LAMMPS 数据文本中的 Masses 部分。

    :param data_text: 原始 LAMMPS 数据文本。
    :return: 修改后的数据文本。
    """
    lines = data_text.splitlines()

    atom_types = None
    # 1. 确定原子类型数
    for line in lines:
        if 'atom types' in line:
            parts = line.strip().split()
            try:
                atom_types = int(parts[0])
                print(f"检测到原子类型数: {atom_types}")
                break
            except (IndexError, ValueError):
                continue

    if atom_types is None:
        print("未能找到 'atom types' 信息。")
        return data_text  # 返回原始文本

    # 2. 定位 'Masses' 部分
    masses_start = None
    for idx, line in enumerate(lines):
        if line.strip().lower() == 'masses':
            masses_start = idx
            print(f"'Masses' 部分开始于第 {masses_start + 1} 行")
            break

    if masses_start is None:
        print("未能找到 'Masses' 部分。")
        return data_text  # 返回原始文本

    # 3. 删除 'Masses' 和其后的 atom_types 行
    # 根据您的原始数据示例，假设 'Masses' 后直接跟 atom_types 行
    # 如果 'Masses' 后有空行，可以调整 masses_end
    # 这里假设 'Masses' 后直接跟 atom_types 行
    masses_end = masses_start + 1 + atom_types + 1
    print(f"删除第 {masses_start + 1} 行到第 {masses_end} 行（共 {atom_types + 1} 行）")
    del lines[masses_start:masses_end]

    # 4. 返回修改后的文本
    modified_text = "\n".join(lines)
    print("已成功删除 'Masses' 部分。")
    return modified_text


def poscar_to_lammps_data(poscar_file, lammps_data_file):
    """
    将 POSCAR 文件转换为 LAMMPS 数据文件，并删除 Masses 部分。

    :param poscar_file: 输入的 POSCAR 文件路径。
    :param lammps_data_file: 输出的 LAMMPS 数据文件路径。
    """
    # 读取 POSCAR 文件
    poscar = Poscar.from_file(poscar_file)
    structure = poscar.structure

    # 创建 LAMMPS 数据对象，atom_style atomic 需要不包含电荷等额外信息
    lammps_data = LammpsData.from_structure(structure)

    # 获取 LAMMPS 数据的字符串表示
    lammps_data_str = lammps_data.get_str()

    # 删除 Masses 部分
    modified_lammps_data_str = remove_masses_section_from_text(lammps_data_str)

    # 写入 LAMMPS 数据文件
    with open(lammps_data_file, 'w', encoding='utf-8') as f:
        f.write(modified_lammps_data_str)

    print(f"已成功生成 LAMMPS 数据文件，保存至: {lammps_data_file}")


if __name__ == "__main__":
    poscar_file = "POSCAR"             # 输入的 POSCAR 文件
    lammps_data_file = "data.lammps"   # 输出的 LAMMPS 数据文件
    poscar_to_lammps_data(poscar_file, lammps_data_file)

检测到原子类型数: 3
'Masses' 部分开始于第 11 行
删除第 11 行到第 15 行（共 4 行）
已成功删除 'Masses' 部分。
已成功生成 LAMMPS 数据文件，保存至: data.lammps


In [2]:
poscar = Poscar.from_file(poscar_file)
structure = poscar.structure

# 创建 LAMMPS 数据对象，atom_style atomic 需要不包含电荷等额外信息
lammps_data = LammpsData.from_structure(structure)

In [12]:
def remove_masses_section(file_path, output_path=None):
    """
    根据原子类型数删除 LAMMPS 数据文件中的 Masses 部分。

    :param file_path: 原始 LAMMPS 数据文件路径。
    :param output_path: 修改后文件的输出路径。如果为 None，则覆盖原文件。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    atom_types = None
    # 1. 确定原子类型数
    for line in lines:
        if 'atom types' in line:
            parts = line.strip().split()
            try:
                atom_types = int(parts[0])
                print(f"Detected atom types: {atom_types}")
                break
            except (IndexError, ValueError):
                continue
    
    if atom_types is None:
        print("未能找到 'atom types' 信息。")
        return
    
    # 2. 定位 'Masses' 部分
    masses_start = None
    for idx, line in enumerate(lines):
        if line.strip().lower() == 'masses':
            masses_start = idx
            print(f"'Masses' section starts at line: {masses_start + 1}")
            break
    
    if masses_start is None:
        print("未能找到 'Masses' 部分。")
        return
    
    # 3. 删除 'Masses' 和其后的 atom_types 行
    masses_end = masses_start + 2 + atom_types
    print(f"Removing lines {masses_start + 1} 到 {masses_end} (共 {atom_types + 1} 行)")
    del lines[masses_start:masses_end]
    
    # 4. 写回文件
    if output_path is None:
        output_path = file_path  # 覆盖原文件
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.writelines(lines)
    
    print(f"已成功更新文件，保存至: {output_path}")

# 使用示例
input_file = 'data.lammps'       # 原始文件路径
output_file = 'data.new.lammps'  # 输出文件路径（可选）

remove_masses_section(input_file, output_file)


Detected atom types: 3
'Masses' section starts at line: 11
Removing lines 11 到 15 (共 4 行)
已成功更新文件，保存至: data.new.lammps
